# Recommenders 3 -- Sequence Recommenders (45m) 

## Goals of this practical:

- Understand the sequence recommendation framework (~5min)
- Load/Format dataset (~5min)
- Understand/train the prod2vec model (~10min)
- Evaluate (~10min)
- Visualize (~10min)
- Fiddle (~5min)



In [1]:
! pip install gensim --upgrade


     ---------------------------------------- 24.0/24.0 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.0
    Uninstalling gensim-4.3.0:
      Successfully uninstalled gensim-4.3.0


In [2]:
import pandas as pd
import numpy as np

# Sequence Recommenders:

> What will you click next ?

The sequence recommendation setting is a particular case of the implicit collaborative filtering setting. Given a sequence of items $i_0,i_1,...,i_n$ the goal is to predict the $i_{(n+1)},...$ items the user will consume. Playlist continuation is a neat use case of sequence recommenders. You've been listening to those songs, what can you listen to now ?


This setting differs from the classical collaborative filtering because the history is the recent trace and not the full saved interactions. Also, it's possible to do sequence recommendation without any specific latent user profile. 

#### Here we propose to explore this unpersonalized sequence recommandation

## Data used : [smallest movie-lens dataset](https://grouplens.org/datasets/movielens/)

Here we'll use the same data as before but instead of seeing $(user,item,rating)$ triplets or a $(user,item)$ interaction , we'll see item sequences: $user: [item, item,...]$

## Loading Data (same as before but in chronological order):

In [4]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings = ratings.sort_values("timestamp",ascending=True)
print(ratings.iloc[0]["timestamp"] < ratings.iloc[-1]["timestamp"] ) # just checking 

ratings.head(5)

# we also load titles and create an id2title dictionnary
titleCSV = pd.read_csv("dataset/movies.csv")
titleCSV.head(5)
id2title = titleCSV[["movieId","title"]].set_index("movieId").to_dict()["title"]

True


## (a) Create sequence datasets:
For this task, we need sequences of items as data:

## (Todo): extract all movie sequences (in chronological order) from the dataset:


In this dataset, each user has seen at least 20 movies.


- We need to extract all movie rating sequences (there is one per user) from the dataset:

`sequence_of_movies = [[movieid,...],[movieid,...],...]`

In [8]:
sequences_of_movies = []# To Complete


user_groups = ratings.groupby('userId')
for user_id, user_ratings in user_groups:
    movie_ids = user_ratings['movieId'].tolist()
    sequences_of_movies.append(movie_ids)
    
sequences_of_movies

[[804,
  1210,
  2628,
  2826,
  2018,
  3578,
  3617,
  3744,
  101,
  441,
  2858,
  1473,
  2997,
  235,
  1060,
  356,
  2700,
  1500,
  223,
  3243,
  2395,
  3253,
  1517,
  1580,
  1732,
  3450,
  2596,
  333,
  543,
  231,
  1042,
  3052,
  216,
  500,
  3809,
  1777,
  3,
  2542,
  2502,
  1080,
  1136,
  1793,
  1256,
  1927,
  923,
  3671,
  260,
  1097,
  1073,
  2872,
  2174,
  2093,
  2005,
  1967,
  2161,
  367,
  2797,
  2193,
  3479,
  2143,
  2105,
  2054,
  2253,
  673,
  3489,
  1009,
  1920,
  3439,
  3440,
  1198,
  1196,
  3793,
  1214,
  2692,
  1197,
  3062,
  2944,
  2028,
  2571,
  2949,
  1222,
  1291,
  1220,
  457,
  3703,
  940,
  110,
  1954,
  2947,
  2948,
  2000,
  6,
  2529,
  2993,
  2427,
  3527,
  3639,
  2991,
  592,
  1275,
  2916,
  1573,
  1408,
  316,
  2406,
  2273,
  2528,
  3441,
  1587,
  480,
  423,
  3729,
  2640,
  733,
  1049,
  2058,
  70,
  3740,
  2366,
  919,
  3168,
  2987,
  1127,
  2414,
  2115,
  590,
  2046,
  349,
  648,
  2

## (Todo): Create a train/test dataset

Here, we propose as task to predict the last 5 items of each sequence.

In [9]:
train_seq,test_seq = [],[]

for seq in sequences_of_movies:
    train_seq.append(seq[:-5])
    test_seq.append(seq[-5:])
    
last_consumed_item = [seq[-1] for seq in train_seq] # We save the last consumed item for each list
                                                    # We'll use it as a starting point

## (Todo): Create the list of the most popular movies

- Here, popular is the number of times the movie appears in a list

In [13]:
from collections import Counter

counts = Counter([item for seq in train_seq for item in seq])
most_popular = [item for item, count in counts.most_common()]

num_items = len(most_popular)

In [15]:
most_popular[:10]

[356, 318, 296, 2571, 593, 260, 480, 110, 589, 1]

``` python
#Most popular looks like this:
[356,318,296,2571,593,260,480,110,589,...]
 ```

## Word2Vec skip-gram <=> Prod2Vec


### Word2Vec

The MAIN idea of word2vec is to maximise the similarity (dot product) between the vectors for words which appear close together (in the context of each other) in text, and minimise the similarity of words that do not. 

This can be applied to products instead of words: it clusters similar products together.


#### Paper Abstract:
> In recent years online advertising has become increasingly ubiquitous and effective. Advertisements shown to visitors fund sites and apps that publish digital content, manage social networks, and operate e-mail services. Given such large variety of internet resources, determining an appropriate type of advertising for a given platform has become critical to financial success. Native advertisements, namely ads that are similar in look and feel to content, have had great success in news and social feeds. However, to date there has not been a winning formula for ads in e-mail clients. In this paper we describe a system that leverages user purchase history determined from e-mail receipts to deliver highly personalized product ads to Yahoo Mail users. We propose to use a novel neural language-based algorithm specifically tailored for delivering effective product recommendations, which was evaluated against baselines that included showing popular products and products predicted based on co-occurrence. We conducted rigorous offline testing using a large-scale product purchase data set, covering purchases of more than 29 million users from 172 e-commerce websites. Ads in the form of product recommendations were successfully tested on online traffic, where we observed a steady 9% lift in click-through rates over other ad formats in mail, as well as comparable lift in conversion rates. Following successful tests, the system was launched into production during the holiday season of 2014

[Prod2Vec Model](https://arxiv.org/abs/1606.07154)



## Gensim has the best python implementation of word2vec's algorithms:

We can just use these raw implementations. The only thing to do is to consider items as words:

In [21]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



train_seq_str = [list(map(str,seq)) for seq in train_seq] # we just say that our items id's are strings..
    

# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                vector_size=50, window=10,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=0.75, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                epochs=50)

2023-03-22 21:41:27,645 : INFO : collecting all words and their counts
2023-03-22 21:41:27,645 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-03-22 21:41:27,666 : INFO : collected 9616 word types from a corpus of 97786 raw words and 610 sentences
2023-03-22 21:41:27,667 : INFO : Creating a fresh vocabulary
2023-03-22 21:41:27,697 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 retains 9616 unique words (100.00% of original 9616, drops 0)', 'datetime': '2023-03-22T21:41:27.697501', 'gensim': '4.3.1', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
2023-03-22 21:41:27,698 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 leaves 97786 word corpus (100.00% of original 97786, drops 0)', 'datetime': '2023-03-22T21:41:27.698502', 'gensim': '4.3.1', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v

2023-03-22 21:41:56,762 : INFO : EPOCH 28: training on 97786 raw words (97664 effective words) took 1.1s, 89992 effective words/s
2023-03-22 21:41:57,844 : INFO : EPOCH 29 - PROGRESS: at 43.11% examples, 44954 words/s, in_qsize 5, out_qsize 1
2023-03-22 21:41:57,889 : INFO : EPOCH 29: training on 97786 raw words (97655 effective words) took 1.1s, 87414 effective words/s
2023-03-22 21:41:58,896 : INFO : EPOCH 30 - PROGRESS: at 46.72% examples, 48360 words/s, in_qsize 5, out_qsize 1
2023-03-22 21:41:58,972 : INFO : EPOCH 30: training on 97786 raw words (97657 effective words) took 1.1s, 90651 effective words/s
2023-03-22 21:42:00,074 : INFO : EPOCH 31 - PROGRESS: at 41.48% examples, 44228 words/s, in_qsize 5, out_qsize 1
2023-03-22 21:42:00,134 : INFO : EPOCH 31: training on 97786 raw words (97661 effective words) took 1.2s, 84882 effective words/s
2023-03-22 21:42:01,153 : INFO : EPOCH 32 - PROGRESS: at 45.74% examples, 47839 words/s, in_qsize 5, out_qsize 1
2023-03-22 21:42:01,210 : IN

### A few things:

In [29]:
print(w2v.wv.vectors[0])              # The vector of index 0
print(w2v.wv.index_to_key[0])           # codes for the movieId 356
print(w2v.wv.index_to_key[356].index)      # Inverse mapping


[-0.2981002   0.0011749  -0.34692243  0.30889645 -0.3732565  -0.33394295
  0.5783879   0.4760714  -0.8009738  -0.29123536 -0.08724707 -0.98687243
  0.04493601  0.5526333  -0.03142472  0.54375523  0.29508746 -0.2099833
 -0.2857274  -0.5990341  -0.12888955  0.2940198   1.0392634  -0.23533235
 -0.09564937 -0.18407422 -0.39240307  0.4982801  -0.0554342  -0.24231
  0.17252384  0.07694855  0.40634722  0.11878206 -0.45771652  0.17122915
 -0.24909337  0.44201496  0.33783585 -0.35293847  0.40892625 -0.00547365
 -0.20845263  0.59116    -0.03563137  0.36038858 -0.15979582 -0.25510815
  0.149237    0.11888317]
356
<built-in method index of str object at 0x000001847BB63EB0>


## Getting similar items:

The heart of the algorithm is in the similar item search. As in word2vec, we simply use cosine distance between items to find "similar items"

### We can search by id's

In [42]:
def get_similar_ids(w2vmodel,iid,num=5):
    
    if str(iid) in w2vmodel.wv.key_to_index:
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(str(iid),topn=num)] 
    else:
        return []

get_similar_ids(w2v,last_consumed_item[0],num=5)

[26142, 66915, 27829, 4577, 26195]

### Or by vector

In [43]:
w2v.wv[str(last_consumed_item[0])]

array([-5.88995777e-02,  1.45996034e+00, -1.98743597e-01,  7.83524156e-01,
        3.13044637e-01, -2.75259227e-01,  8.79448056e-01,  1.03722811e+00,
        2.67736346e-01, -1.14326239e-01, -2.66539872e-01, -5.75336337e-01,
       -2.74461627e-01,  1.22779876e-01,  3.89361866e-02, -7.17599511e-01,
        5.56412973e-02,  9.19748545e-01, -1.20650589e+00, -7.00136483e-01,
        7.23808110e-01,  8.74374658e-02,  1.07511830e+00, -7.88493872e-01,
       -5.88973880e-01,  2.52909362e-01, -3.77090573e-02,  4.28691506e-01,
       -7.37186790e-01,  3.46313827e-02, -7.03731477e-01,  8.13317955e-01,
       -2.22672403e-01,  1.15540624e+00, -1.86190754e-01,  5.57403684e-01,
       -1.70856237e-01, -5.75757623e-02, -7.17231393e-01, -7.09470332e-01,
       -7.52767995e-02, -7.75638386e-04, -5.61495006e-01,  1.47060275e+00,
        7.14774370e-01, -5.87676205e-02,  5.87647736e-01, -1.29374266e+00,
        8.39188620e-02, -8.58158842e-02], dtype=float32)

In [44]:
def get_similar_vectors(w2vmodel,vec,num=5):
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(positive=[vec],topn=num)] 

get_similar_vectors(w2v,w2v.wv[str(last_consumed_item[0])],num=5) # items are strings

[157, 26142, 66915, 27829, 4577]

### Let's see if this works

We can query by id

In [46]:
ID = 1
NUM_SIM = 3

print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_ids(w2v,ID,NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Toy Story (1995)

-->  Lion King, The (1994)
-->  Forrest Gump (1994)
-->  Beauty and the Beast (1991)


We can also query by vector

**NOTE:** the 1st results can be the item(s) you've used to query

In [47]:
ID = 1
NUM_SIM = 4
print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_vectors(w2v,w2v.wv[str(ID)],NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Toy Story (1995)

-->  Toy Story (1995)
-->  Lion King, The (1994)
-->  Forrest Gump (1994)
-->  Beauty and the Beast (1991)


Our result was the following (for ID = 1 & NUM_SIM = 3)

>Movies similar to:  Toy Story (1995)
>-  Beauty and the Beast (1991)
-   Toy Story 2 (1999)
-   Lion King, The (1994)


Using vectors enables operations like additions to be made

In [48]:
ID1 = 2571
ID2 = 589
NUM_SIM = 10

vec = np.max([w2v.wv[str(ID1)],w2v.wv[str(ID2)]],axis=0)# + w2v.wv[str(318)]

print("Movies similar to: ", id2title[ID1] , "+",  id2title[ID2] )
print("")
for x in get_similar_vectors(w2v,vec ,NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Matrix, The (1999) + Terminator 2: Judgment Day (1991)

-->  Matrix, The (1999)
-->  Terminator 2: Judgment Day (1991)
-->  Terminator, The (1984)
-->  Saving Private Ryan (1998)
-->  Seven (a.k.a. Se7en) (1995)
-->  Sixth Sense, The (1999)
-->  Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
-->  Star Wars: Episode V - The Empire Strikes Back (1980)
-->  Andrew Dice Clay: Dice Rules (1991)
-->  Fuzz (1972)


Ok, we now have a good base for our sequence recommendation algorithm, let's write something to evaluate our predictions

## (Todo) write a `get_relevance_list(proposed_ids,real_ids)` function:

This function will be used to compare proposed items w/ real items:


- A relevant item is an item which is in the ground truth
- It returns a list which length is the number of proposed items filled of 0's and 1's : 0 means the item is not relevant, 1 means it's relevant.

- get_relevance_list([1,2,3,4],[1,4,5,6]) should returns [1,0,0,1]  because items 1 and 4 are relevant.


In [55]:
def get_relevance_list(proposed_ids,real_ids):
    real_ids = set(real_ids)
    return [1 if i in real_ids else 0 for i in proposed_ids]
get_relevance_list([1,2,3,4],[1,4,5,6]) #returns [1,0,0,1]

[1, 0, 0, 1]

### Let's test our function on our data

In [56]:
get_relevance_list(most_popular[:25],test_seq[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [57]:
get_relevance_list(get_similar_ids(w2v,last_consumed_item[0],25),test_seq[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

## Ok, now, let's write prediction funtions:

- `predict_pop` will recommend the k's most popular items
- `predict_w2v` will recommend the k's most similar items to the last one consumed

#### (TODO) : complete those functions

In [58]:
def predict_pop(last_seen,k):
    return most_popular[:k]

def predict_w2v(last_seen,k):
    return get_similar_ids(w2v,last_consumed_item[0],k)

#data is list of last_consumed:
def get_predictions(predict_func,data,truth,k=5):
    if k == -1 or k == 0:
        k = num_items
    return [get_relevance_list(predict_func(last_seen,k),will_see) for last_seen,will_see in zip(data,truth)]

**Note**: The `get_predictions(...)` function returns the relevant list associated to predictions

### The following cells should return list of lists

In [59]:
print(get_predictions(predict_pop,last_consumed_item[:5],test_seq[:5],3))
print(get_predictions(predict_w2v,last_consumed_item[:5],test_seq[:5],3))

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


expected output: 
```
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

```

## The return of the MRR and nDCG functions

In [60]:
test_list = [[0,0,1],[0,1,0],[1,0,0],[0,0,0]]

def rr(list_items):
    relevant_indexes = np.asarray(list_items).nonzero()[0]
    
    if len(relevant_indexes) > 0:
        return 1/(relevant_indexes[0]+1) # arrays are indexed from 0
    else:
        return 0

def mrr(list_list_items):
    return np.mean([rr(list_item) for list_item in list_list_items])

mrr(test_list) #0.4583333333333333

# The dcg@k is the sum of the relevance, penalized gradually
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        
    return 0.

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# dcg_at_k(r, 1) => 3.0
# dcg_at_k(r, 2) => 4.2618595071429155
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
print(dcg_at_k(r, 1))
print(dcg_at_k(r, 2))

def mean_dcg(rel_lists,k):
    return np.mean([dcg_at_k(rel_list,k) for rel_list in rel_lists])

# And it's normalized version
def ndcg_at_k(r, k):
    """
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    """
    dcg_max =  dcg_at_k(sorted(r)[::-1],k) 
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# ndcg_at_k(r, 1) => 1.0
# ndcg_at_k(r, 4) => 0.794285
    
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]    
ndcg_at_k(r, 4)

def mean_ndcg(rel_lists,k):
    return np.mean([ndcg_at_k(rel_list,k) for rel_list in rel_lists])

3.0
4.2618595071429155


## Let's see how this naïve way of predicting items to show works

In [61]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

print("1/MRR")
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))
print("")
print("DCG")
print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))
print("")
print("nDCG")
print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

1/MRR
18.089018857528632
405.4749844882932

DCG
0.0507673354188168
0.0

nDCG
0.01745016484461114
0.0


### (TODO) Can we do better ?

Now, try a different strategy: 


- History should be discarded from prediction
- Instead of basing the prediction on the last seen item, we'll take all the `seen[-n:]` ones (horizon) into account
- To aggregate all items, we'll simply take the min rank to take into account the history offset.
- Equal scores can be handled using the history offset.
Example: 

> Let's say you chose to use the two last seen items `[item 44, Item 398]` to predict the following items

Therefore, using `get_similar_ids` method on both items will yield two lists of **similar** ranked item id's:
 - Similar to item 44: `[item 1, item 33, item 5]`
 - Similar to item 398: `[item 25, item 1, item 5]`
 scores (rank,offset):
 ```
 scores := {item 1: (0,0), item 33: (1,0), item 5: (2,0) , item 25: (0,1)}```
 
 
Then, aggregation by best rank should yield: `[1,25,33,5]`


In [62]:
def predict_max_w2v(seen,k,horizon=2):
    
    horizon_seen = seen[-horizon:]
    scores = {}
    
    for item_id in horizon_seen:
        similar_items = get_similar_ids(w2v,item,k)
    
    # To complete
    
    return # a list


w2v_best_preds = get_predictions(predict_max_w2v,train_seq,test_seq,-1)

print(1/mrr(w2v_max_preds))
print(mean_dcg(w2v_max_preds,5))
print(mean_ndcg(w2v_max_preds,5))

TypeError: 'NoneType' object is not iterable

#### => Not really better

## Let's visualize learned embeddings

Just like in the 1st practical, we propose to visualize learnt items embeddings with the [Tensorflow projector](https://projector.tensorflow.org/).

In [63]:
# This function saves embeddings (a numpy array) and associated labels into tsv files.

def save_embeddings(embs,dict_label,path="saved_word_vectors"):
    """
    embs is Numpy.array(N,size)
    dict_label is {str(word)->int(idx)} or {int(idx)->str(word)}
    """
    def int_first(k,v):
        if type(k) == int:
            return (k,v)
        else:
            return (v,k)

    np.savetxt(f"{path}_vectors.tsv", embs, delimiter="\t")

    #labels 
    if dict_label:
        sorted_labs = np.array([lab for idx,lab in sorted([int_first(k,v) for k,v in dict_label.items()])])
        print(sorted_labs)
        with open(f"{path}_metadata.tsv","w") as metadata_file:
            for x in sorted_labs: #hack for space
                if len(x.strip()) == 0:
                    x = f"space-{len(x)}"
                    
                metadata_file.write(f"{x}\n")

In [66]:
vec2title = {i:id2title[int(mid)] for i,mid in enumerate(w2v.wv.index_to_key)}

In [67]:
save_embeddings(w2v.wv.vectors,vec2title)

['Forrest Gump (1994)' 'Shawshank Redemption, The (1994)'
 'Pulp Fiction (1994)' ...
 'Weekend (a.k.a. Le Week-end) (Week End) (1967)' 'Mischief (1985)'
 'Splinter (2008)']


## How to:

- Now, [open this link](https://projector.tensorflow.org/), and select "load".
- look for saved_word_vectors_vectors.tsv and saved_word_vectors_metadata.tsv. 

=> These are respectively, the items latent representations and their labels

## Hyperparameters matter when using Word2Vec for Item recommendation:


> Skip-gram with negative sampling, a popular variant of Word2vec originally designed and tuned to create word embeddings for Natural Language Processing, has been used to create item embeddings with successful applications in recommendation. While these fields do not share the same type of data, neither evaluate on the same tasks, recommendation applications tend to use the same already tuned hyperparameters values, even if optimal hyperparameters values are often known to be data and task dependent. We thus investigate the marginal importance of each hyperparameter in a recommendation setting through large hyperparameter grid searches on various datasets. Results reveal that optimizing neglected hyperparameters, namely negative sampling distribution, number of epochs, subsampling parameter and window-size, significantly improves performance on a recommendation task, and can increase it by an order of magnitude. Importantly, we find that optimal hyperparameters configurations for Natural Language Processing tasks and Recommendation tasks are noticeably different. 

[Hyperparameters matter](https://arxiv.org/abs/1804.04212)

#### It turns out that  hyperparameters are really important for this task: especially the sampling parameter.  Try and learn multiple models to see how the ns_exponent parameter modifies the results:


In [71]:
# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                vector_size=50, window=3,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=-0.4, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                epochs=25)



2023-03-22 22:16:20,738 : INFO : collecting all words and their counts
2023-03-22 22:16:20,741 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-03-22 22:16:20,767 : INFO : collected 9616 word types from a corpus of 97786 raw words and 610 sentences
2023-03-22 22:16:20,768 : INFO : Creating a fresh vocabulary
2023-03-22 22:16:20,813 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 retains 9616 unique words (100.00% of original 9616, drops 0)', 'datetime': '2023-03-22T22:16:20.812241', 'gensim': '4.3.1', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
2023-03-22 22:16:20,815 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 leaves 97786 word corpus (100.00% of original 97786, drops 0)', 'datetime': '2023-03-22T22:16:20.815231', 'gensim': '4.3.1', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v

In [72]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

In [73]:
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))

print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))

print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

18.089018857528632
289.8393404804183
0.0507673354188168
0.0007060271443826116
0.01745016484461114
0.00023945631121755155


## Still got time ? Try making a more clever item selection mechanism:

- You could, for example, cluster items in groups (using k-means) and propose the most popular items of the last seen group